In [77]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../src")
import utils.munging_utils as query
import utils.preprocessing_utils as process
from sklearn.utils import resample
from sklearn import metrics
import pandas as pd
import numpy as np
import warnings
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split
warnings.simplefilter("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction

This notebook will explore how models performed on our metadata through bootstrapped subsamling

In [54]:
data = query.get_file_entity("syn21281633")
data = data[(data["version"] == "mpower_v1") | (data["version"] == "mpower_v2")]

METADATA_COLS = ["PD", "appVersion", "phoneInfo", "age", "gender"]

def one_hot_encoder(data, *features):
    for feature in features:
        OHE_data = pd.get_dummies(data[feature], drop_first = True, dtype = float)
        data = pd.concat([data, OHE_data], axis = 1).drop(feature, axis = 1)
    return data

In [65]:
metadata = data[METADATA_COLS]
metadata = one_hot_encoder(metadata, "appVersion", "phoneInfo", "gender")

In [70]:
## set X_test, y_test as OOB samples
## set X_train, y_train as sample for bootstrap
X_train, X_test, y_train, y_test = train_test_split(metadata, metadata["PD"], test_size=0.25, random_state=100)

In [71]:
## shape of c
X_train.shape

(78130, 220)

In [72]:
X_test.shape

(26044, 220)

In [74]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [82]:
print(metrics.classification_report(y_test, rf.predict(X_test)))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      7335
         1.0       1.00      1.00      1.00     18709

    accuracy                           1.00     26044
   macro avg       1.00      1.00      1.00     26044
weighted avg       1.00      1.00      1.00     26044



In [79]:
y_test

46873     1.0
18849     1.0
101762    1.0
94153     0.0
71618     1.0
43753     1.0
51665     1.0
62369     1.0
102433    1.0
44595     1.0
99263     1.0
21497     1.0
82167     1.0
68527     1.0
54804     1.0
52976     1.0
80818     0.0
95403     1.0
67825     1.0
57201     1.0
61719     1.0
89025     1.0
28759     0.0
68327     1.0
91543     0.0
97033     1.0
36937     1.0
90846     1.0
13421     0.0
53029     1.0
         ... 
102219    1.0
24238     1.0
25236     0.0
28902     0.0
87299     0.0
28466     0.0
93876     0.0
83630     1.0
94117     0.0
75458     0.0
102034    1.0
49930     1.0
87739     1.0
31651     1.0
52864     1.0
12011     0.0
82650     1.0
42925     0.0
31551     0.0
59939     1.0
43624     0.0
95004     1.0
88773     1.0
49770     1.0
38870     1.0
69790     0.0
97176     1.0
78411     1.0
80211     0.0
1153      1.0
Name: PD, Length: 26044, dtype: float64

In [80]:
rf.predict(X_test)

array([1., 1., 1., ..., 1., 0., 1.])